#### 필요 라이브러리 Import 

In [4]:
from selenium import webdriver
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import time
import re
import unicodedata
import requests

#### 드라이버 기본 설정 

In [5]:
def set_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_experimental_option("excludeSwitches", ["enable-logging"])

    driver = webdriver.Chrome("./chromedriver.exe", options=options)
    driver.set_window_size(1024, 768)
    return driver 

set_driver()

#### Instagram 접속 

In [13]:
#인스타그램 페이지 연결
driver.get('https://www.instagram.com')
time.sleep(2)

#### Instagram login

In [14]:
email = input('이메일을 입력하세요.')
pw = input('비밀번호를 입력하세요. 결과 창에 비밀번호가 공개되는 것을 방지하려면 아무것도 입력하지 마세요.\
            생성된 크롬창에서 직접 비밀번호를 입력하세요.')

# id 입력 
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

# pw 입력 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(pw)
input_pw.submit()

print('인스타그램 로그인 확인을 위한 10초 타이머 시작')
for i in tqdm(range(10)):   # 로그인 확인을 위한 대기시간 10초 
    time.sleep(1)

이메일을 입력하세요.pomate@g.skku.edu
비밀번호를 입력하세요. 결과 창에 비밀번호가 공개되는 것을 방지하려면 아무것도 입력하지 마세요.            생성된 크롬창에서 직접 비밀번호를 입력하세요.Dhwoehd12
인스타그램 로그인 확인을 위한 10초 타이머 시작


  0%|          | 0/10 [00:00<?, ?it/s]

#### 해시태그 기반 게시글 검색 

In [7]:
def insta_searching(word):
    '''
    검색하고 싶은 해시태그를 입력으로 받아, url을 반환
    '''
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [8]:
url = insta_searching('제주도맛집')
url

'https://www.instagram.com/explore/tags/제주도맛집'

In [174]:
word = input('키워드를 입력해주세요: ')    # 검색어
url = insta_searching(word)   # url 생성 
driver.get(url)   # 접속 
time.sleep(10)

키워드를 입력해주세요: 홍대


#### 게시글 크롤링하기

In [7]:
def select_first(driver):
    '''
    해시태그 결과 게시글 중 첫번째 게시물을 선택하는 함수
    '''
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]').click()
    time.sleep(3)

In [8]:
def get_content(driver):
    '''
    크롤링할 콘텐츠 지정 
    본문 내용, 좋아요 수, 위치, 해시태그, 이미지 url을 크롤링해서 반환함 
    '''
    # 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    try:   # 본문 내용 가져오기
        content = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]\
        /ul/div/li/div/div/div[2]/div[1]/span').text 
        print(content)
        # content = unicodedata.normalize('NFC', content) 
    except:
        content = ' x'
    # 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    try:   # 좋아요 수 가져오기  
        like = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/section[2]/\
        div/div/div/a/div/span').text    
    except:
        like = 0
    try:   # 위치정보 가져오기
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    try:   # img url 가져오기 /html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[1]/div[1]/img
        img_data = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div/article/div/div[1]/div/div/div[1]/img')
        img_url = img_data.get_attribute('src')
    except:
        img_url = ''
    # 수집한 정보 저장하기
    data = [content, like, place, tags, img_url]
    return data

In [9]:
def get_n_contents(driver, target):
    word = input('키워드를 입력해주세요: ')    # 검색어
    url = insta_searching(word)   # url 생성 
    driver.get(url)   # 접속 
    time.sleep(10)
    select_first(driver)
    time.sleep(3)
    results = []   # 크롤링 결과 저장할 변수  
    
    for idx in tqdm(range(1, int(target) + 1)):
        try:
            data = get_content(driver)    # 게시글 정보 가져오기
            results.append(data)
            if idx == 1:
                driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button').click()
                time.sleep(1)
            else:
                driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]/button').click()
                time.sleep(1)
        except:
            continue 
    return word, results

In [15]:
# 50개 게시글 크롤링 
keyword, results = get_n_contents(driver, 500)

키워드를 입력해주세요: 커플


  0%|          | 0/500 [00:00<?, ?it/s]

울애기 얼집에서 생파🧡
집에서 찍은것보다 더 예쁘게 잘 나왔네? ㅋㅋㅋㅋ 친구들이 축하해줘서 너무너무 행복했다는 하찌니❣️

하지니가 즐겁고 행복했다니 엄마두 좋다🧡

울애기 다시한번 엄마아빠한테 와줘서 고마워❣️사랑해🥰

#24개월아기#두돌#생일#두돌생일파티#아들룩#아들맘#초보맘#도치맘#쥐띠맘#얼집생파#럽스타그램#육아#소통#일상

#내일 캠핑가서 또 생파하자…..ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ😅

정말 1주일내내 생일이라더니 그말이 맞구나🤣
유채꽃이 내 키랑 같다니..!!😳
끝물이라서 아쉬웠지만
그래도 너무 이뻣던 유채꽃💛

#제주도
#신혼여행
#가파도유채꽃
둘이 첫노래방🤭
신랑 노래는 나만 듣는걸로🙈🙊🙉
신난다 신나
.
.
.
.
#럽스타그램
#신혼부부일상
#평일일상
#저녁마실
#화성우리집
저녁산책

#일상그램 #좋아요 #육아맘 #육아스타그램 #맘스타그램 #행복스타그램 #럽스타그램 #육아소통 #육아일기 #ootd #딸바보 #엄마랑 #소통여행 #여행스타그램 #가족스타그램 #운동스타그램 #힐링 #옷스타그램 #쇼핑스타그램 #셀카그램 #사랑해 #운동스타그램 #수영 #집스타그램 #금요일 #저녁

울애기 얼집에서 생파🧡
집에서 찍은것보다 더 예쁘게 잘 나왔네? ㅋㅋㅋㅋ 친구들이 축하해줘서 너무너무 행복했다는 하찌니❣️

하지니가 즐겁고 행복했다니 엄마두 좋다🧡

울애기 다시한번 엄마아빠한테 와줘서 고마워❣️사랑해🥰

#24개월아기#두돌#생일#두돌생일파티#아들룩#아들맘#초보맘#도치맘#쥐띠맘#얼집생파#럽스타그램#육아#소통#일상

#내일 캠핑가서 또 생파하자…..ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ😅

정말 1주일내내 생일이라더니 그말이 맞구나🤣


#### 크롤링한 데이터 csv 파일로 저장 

In [212]:
import pandas as pd
import os 

if not os.path.exists('./files'):
    os.mkdir('./files')

In [213]:
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'like', 'place', 'tags', 'img_url']
results_df.drop_duplicates(subset = 'content' , inplace = True)   # 중복 게시글 제거
results_df.reset_index(drop=True, inplace = True)   # 인덱스 재설정
results_df.to_csv(f'./files/{keyword}.csv', index=None)